In [1]:
from kafka import KafkaProducer
import json
import time
import random
from math import cos, pi

In [2]:
def generate_sensor_data():
    timestamp = int(time.time())

    # Simulate IoT sensor data for water quality metrics with realistic patterns
    water_temperature = random.uniform(1, 3) + round(20 + 5 * (1 + 0.5 * (1 + cos((timestamp % 86400) / 86400 * 2 * pi))), 2)
    ph_level = random.uniform(0, 1) + round(7.5 + 0.2 * (1 + cos((timestamp % 86400) / 86400 * 2 * pi)), 2)
    turbidity = round(random.uniform(5, 50), 2)  # Turbidity in NTU (Nephelometric Turbidity Units)
    dissolved_oxygen = round(random.uniform(5, 12), 2)  # Dissolved Oxygen in mg/L

    return {
        "timestamp": timestamp,
        "water_temperature": water_temperature,
        "ph_level": ph_level,
        "turbidity": turbidity,
        "dissolved_oxygen": dissolved_oxygen
    }

In [3]:
# Kafka configuration
kafka_topic = "water_quality"
kafka_bootstrap_servers = ["localhost:9092"] 

producer_number = 7
producers = []
producer_ids = []
# Create Kafka producer
for i in range(producer_number):
    producer_ids.append(str(i))
    producers.append(KafkaProducer(
        bootstrap_servers=kafka_bootstrap_servers,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )
)
    
print(f"Producing messages to Kafka topic '{kafka_topic}'...")

try:
    while True:
        for i in range(producer_number):
            # Generate sensor data
            sensor_data = generate_sensor_data()
    
            # Publish sensor data to Kafka
            producers[i].send(kafka_topic, value = sensor_data, key = producer_ids[i].encode('utf-8'))
    
            print(f"Sent: {sensor_data} Producer key: {producer_ids[i]}")

            # Wait for 1 second
            time.sleep(1)
except KeyboardInterrupt:
    print("Stopped producing messages.")
finally:
    for producer in producers:
        producer.close()

Producing messages to Kafka topic 'water_quality'...
Sent: {'timestamp': 1741192688, 'water_temperature': 29.047912734799443, 'ph_level': 7.645161503397677, 'turbidity': 5.79, 'dissolved_oxygen': 8.59} Producer key: 0
Sent: {'timestamp': 1741192689, 'water_temperature': 28.033067418572493, 'ph_level': 7.810205691028737, 'turbidity': 7.44, 'dissolved_oxygen': 7.51} Producer key: 1
Sent: {'timestamp': 1741192690, 'water_temperature': 28.453307150752053, 'ph_level': 8.02617302635067, 'turbidity': 44.41, 'dissolved_oxygen': 7.92} Producer key: 2
Sent: {'timestamp': 1741192691, 'water_temperature': 29.493688199284005, 'ph_level': 8.415197111927561, 'turbidity': 21.28, 'dissolved_oxygen': 7.45} Producer key: 3
Sent: {'timestamp': 1741192692, 'water_temperature': 28.87711460109248, 'ph_level': 8.22179374345716, 'turbidity': 41.34, 'dissolved_oxygen': 9.4} Producer key: 4
Sent: {'timestamp': 1741192693, 'water_temperature': 28.767757629496217, 'ph_level': 8.019071952778631, 'turbidity': 31.86,